<h1 align="center">Práctica 5. El problema del ensamblaje (Voraces - Ramificación y Poda)</h1>
<h3 style="display:block; margin-top:5px;" align="center">Algorítmica</h3>
<h3 style="display:block; margin-top:5px;" align="center">Grado en Ingeniería Informática</h3>
<h3 style="display:block; margin-top:5px;" align="center">2024-2025</h3>    
<h3 style="display:block; margin-top:5px;" align="center">Universitat Politècnica de València</h3>
<br>

**Pon/poned aquí tú/vuestros nombre(s):**
- (sustituir por el nombre)

## Índice
1. ### [El problema del ensamblaje](#introduccion)
1. ### [Actividad 1: Solución voraz](#actividad1)
1. ### [Actividad 2: Ramificación](#actividad2)
1. ### [Actividad 3: Calcular estadísticas](#actividad3)
1. ### [Código a completar](#codigo)


<a id='introduccion'></a>

# El problema del ensamblaje

Se trata del problema descrito en los apuntes de teoría en la sección 9.4. Lo mejor es ir directamente al pdf para ver la descripción del problema, pero resumimos aquí algunos datos:

- Hay que ensamblar un total de $M$ piezas con el menor coste posible.
- El coste de ensamblar la pieza $i$ depende del número de piezas ya ensambladas.
- Los datos de entrada se resumen en una matriz `costes` de tamaño $M \times M$ con valores positivos (no hace falta que sean enteros). El valor `costes[i,j]` representa el coste de situar la pieza `i` (un identificador entre `0` y `M-1`) cuando ya se han ensamblado `j` piezas.
- Las soluciones son tuplas de la forma $(x_0,x_1,\ldots,x_{M-1})$ donde $x_i$ es el nº piezas ya montadas en el momento en que se decide montar la pieza que identificamos con el índice $i$.
- La función objetivo es: $f((x_0,x_1,\ldots,x_{M-1})) = \sum_{0 \leq i < M} \mbox{costes}[i,x_i]$
- Todas las permutaciones serían factibles, se trata de encontrar una que corresponda a un coste mínimo (podría haber empates).
- Se trata de un problema conocido en teoría de grafos, el [Problema de la asignación](https://es.wikipedia.org/wiki/Problema_de_la_asignaci%C3%B3n) o [Assignment problem](https://en.wikipedia.org/wiki/Assignment_problem) para el que existen algoritmos como [Kuhn Munkres](https://en.wikipedia.org/wiki/Hungarian_algorithm) con un coste polinómico ($O(|V|^3)$).

## Generación de instancias

Para generar instancias concretas para una talla dada, vamos a recurrir a la generación de números aleatorios utilizando la siguiente función de la biblioteca `numpy`:

In [11]:
import numpy as np
def genera_instancia(M, low=1, high=1000):
    return np.random.randint(low=low,high=high,
                             size=(M,M),dtype=np.int32)

In [12]:
costes = genera_instancia(4,high=10)
costes

array([[6, 3, 1, 6],
       [2, 6, 7, 6],
       [8, 1, 4, 3],
       [2, 7, 3, 8]], dtype=int32)

Con una matriz como ésta (cada vez que lo ejecutes dará normalmente otra distinta):

```python
array([[7, 3, 7, 2],
       [9, 9, 4, 1],
       [9, 4, 8, 1],
       [3, 4, 8, 4]])
```

el coste de ensamblar la pieza 0 en la cuarta posición (después de haber ensamblado 3 piezas) es `costes[0,3]` que vale `2`.


## Representación de los estados
Un estado intermedio $(x_0,x_1,\ldots,x_{k-1})$ se representará mediante una lista Python con esos mismos valores. El estado inicial será la lista vacía `[]`. Un estado solución será una lista de talla $M$.

## Cota optimista

Vamos a utilizar como cota optimista (cota inferior en este caso, pues es un problema de minimización) la suma de:

- La parte conocida.
- Una estimación optimista del coste de situar las piezas que faltan.

Concretamente, la estimación utilizada es una de las descritas en los apuntes de teoría que permite su cálculo de forma incremental:

> Suponer que para cada pieza que queda por ensamblar se selecciona la posición en la que cueste menos ensamblar, **sin importar que esa posición ya haya sido utilizada.**.


$optimistic((x_0,x_1,\ldots,x_{k-1})) = \sum_{0\leq i<k} costes[i,x_i] + \sum_{k  \leq i < M} min_{0 \leq j < M} costes[i,j] $

Es decir, completamos la parte desconocida con el menor coste posible para cada una de las piezas que quedan por ensamblar.

Obviamente, podemos precalcular esos mínimos al inicio para reutilizarlos a lo largo del algoritmo.

Esta cota se puede actualizar **de forma incremental** (teniendo en cuenta el valor de la cota del padre) al ramificar un estado.

> **Nota:** Esta cota **NO** es la más informada para podar por cota optimista. Existe otra más informada consistente en no tener en cuenta los instantes utilizados por las piezas que ya forman parte de la solución parcial. El problema de esta cota alternativa es que resulta más cara de calcular.


## Almacenamiento del conjunto de estados activos

El conjunto de estados activos será una cola de prioridad implementada mediante un *minheap*. Vamos a utilizar una biblioteca estándar de python llamada `heapq`. El siguiente código ilustra las 2 funciones básicas para utilizar un `heapq` como cola de prioridad.

Observa que guardamos tuplas donde el primer campo es el *score* para que así se ordenen por *score* de menor a mayor (nos sirve porque estamos en un problema de **minimización**):

In [13]:
import heapq

A = [] # conjunto vacío de estados activos, es una lista Python normal y corriente
for score,s in [(10,[1]),(3,[2]),(100,[0,1])]:
    heapq.heappush(A,(score,s)) # insertar i en la cola de prioridad A
print(A) # no sale ordenado necesariamente, es un minheap...
while len(A)>0:
    score,s = heapq.heappop(A) # extraer el menor elemento de la cola de prioridad A
    print(score,s)

[(3, [2]), (10, [1]), (100, [0, 1])]
3 [2]
10 [1]
100 [0, 1]


## Esquema de ramificación y poda, funciones auxiliares

Podemos implementar el esquema de ramificación y poda o *branch and bound* de dos formas alternativas e igualmente válidas (entre otras formas más):

- Utilizar una clase y métodos (como en los apuntes de teoría).
- Utilizar una función y poner dentro otras funciones (clausuras o *closure* en inglés).

Vamos a optar por esta segunda opción ya que básicamente podemos ver el problema del ensamblaje como una función que recibe la matriz de costes y nos devuelve la mejor solución encontrada (es decir, tiene un punto de entrada y uno de salida).

Las funciones auxiliares utilizadas son:

- `greedy_solution` calcula una solución *arbitraria* para inicializar la mejor solución en curso. Esto es importante para empezar a podar tan pronto como sea posible. Aunque serviría "cualquier" solución (ejemplo: `[0,1,2,...,M-1]`) cuanto mejor sea esta primera solución, tanto mejor. La restricción es que no debe ser demasiado costosa de calcular. Veremos varias formas de implementar una solución voraz.

- `branch` recibe una solución parcial (un estado intermedio) y su score asociado y va generando soluciones hijas (y su respectivos score) consistentes en añadir un nuevo $x_k$. Se trata de determinar cuántas piezas se han ensamblado en el momento de ensamblar la pieza $k$-ésima. El score asociado **se debe calcular de manera incremental**.

- `is_complete` se limita a decir si una solución parcial es solución o estado terminal. Se proporciona ya implementada puesto que es tan fácil como ver si la longitud de la solución parcial es igual a $M$.

El bucle principal de ramificación y poda es como sigue:

```python
    ...
    while len(A)>0 and A[0][0] < fx:
        s_score, s = heapq.heappop(A)
        for child_score, child in branch(s_score, s):
            if is_complete(child): # si es terminal
                # es factible (pq branch solo genera factibles)
                # falta ver si mejora la mejor solucion en curso
                if child_score < fx:
                    fx, x = child_score, child
            else: # no es terminal
                # lo metemos en el cjt de estados activos si supera
                # la poda por cota optimista:
                if child_score < fx:
                    heapq.heappush(A, (child_score, child) )
    return x,fx
```

Observa que se trata de la estrategia conocida como **poda implícita**:

- Cuando se encuentra una solución que mejora a la mejor hasta el momento se actualiza la mejor solución pero no se revisa el conjunto de estados activos `A`. Es decir, `A` puede contener soluciones que se podrían eliminar con una poda pero que cuando se introdujeron en su momento no se podaron porque el rasero o criterio era distinto. La poda explícita (no la utilizamos en esta práctica) aprovecharía este momento para eliminar esas soluciones podables.

- Por otra parte, no basta con poner `while len(A)>0` sino que hace falta añadir `and A[0][0] < fx:` porque incluso sin vaciar el conjunto de estados activos si sacamos una solución peor que `fx` (score de la mejor solución hasta el momento) todas las que queden en `A` también son podables y no tiene interés procesarlas. Esto NO haría falta con poda explícita.

A continuación mostramos las 3 actividades a realizar y posteriormente el código donde hay que implementar o codificar esas actividades.

<a id='actividad1'></a>

# Actividad 1: Solución voraz

Se trata de obtener una solución con la que inicializar la variable `x` (mejor solución encontrada hasta el momento) y su score correspondiente `fx`.

En el problema del ensamblaje es trivial obtener una solución porque cualquier permutación de índices entre `0` y `M-1` es una solución válida. Es decir, podríamos hacer algo así:

```python
def ensamblaje(costes,
               verbosity=1):

    ...
    
    def naive_solution():
        score, solution = 0, []
        for i in range(M):
            solution.append(i)
            score += costes[i,i]
        return score,solution
```

Es decir, cada $x_i$ es igual a $i$ (los objetos se ponen en orden 0,1,2,...).

Pero cuanto mejor sea la solución inicial antes empezaremos a podar mejor, sin llegar al extremo de calcularla de forma exacta porque, en ese caso ¿para qué usar luego ramificación y poda?

En esta primera actividad debes completar las siguentes funciones `greedy_solution`:

1. Ir por orden pieza a pieza (fila $i$ de la matriz) y elegir el valor $x_i$ (el momento de colocación de esa pieza) que resulte más barato de los que siguen disponibles. Es decir, ir fila por fila de la matriz y elegir (para esa fila) la columna menor de las columnas previamente no elegidas.
2. Ir por orden instante a instante (columna de la matriz) y elegir para cada una la pieza (fila de la matriz) que sea más barata de colocar en ese instante (de entre las piezas que queden por ensamblar).
3. Ordenar de menor a mayor todos valores de la matriz de costes recordando sus coordenadas. Después se recorre utilizando los valores que correspondan a piezas e instantes válidos (descartando el resto) hasta haber situado todas las piezas.
4. Cualquier combinación de los anteriores (se calculan soluciones con varios algoritmos y nos quedamos con la mejor solución).

Debes implementar las funciones voraces y copiarlas en la función principal.

# NOTA: Copia las soluciones voraces de la práctica 4 en el código de la Actividad 3

<a id='actividad2'></a>

# Actividad 2: Función de ramificación

Se trata de completar la función `branch` que recibe 2 argumentos:

- El score del estado padre.
- El estado padre $[x_0,x_1,\ldots,x_{k-1}]$.

Podemos asumir que dicha lista tiene una longitud menor a `M` porque sólo se utiliza `branch` si el estado no es terminal.

Esta función puede hacer una de estas dos cosas:

- Ir devolviendo los estados hijos usando `yield`.
- Generar una lista de los estados hijos y devolver dicha lista.

Ambas aproximaciones son válidas en la medida en que ambas se pueden utilizar desde la función principal en el bucle que hemos descrito arriba y que repetimos a continuación:

```python
    ...
    while len(A)>0 and A[0][0] < fx:
        s_score, s = heapq.heappop(A)
        for child_score, child in branch(s_score, s):
            ...
```

Es importante calcular el valor `child_score` de manera **incremental** a partir del valor `s_score`. Para ello, observa que preprocesamos el mínimo coste de ensamblar cada pieza en un vector llamado `minCoste`.

<a id='actividad3'></a>

# Actividad 3: Obtención de estadísticas

En esta parte básicamente debes añadir unos contadores en el cuerpo principal de "Ramificación y Poda" para calcular las siguientes cosas:

- La variable `iterations` cuenta el nº de iteraciones del bucle `while` principal.
- La variable `maxA` contabiliza el tamaño máximo que ha llegado a alcanzar el conjunto de estados activos a lo largo de la ejecución.
- `gen_states` cuenta el número total de estados generados por `branch` a lo largo de toda la ejecución.
- `podas_opt` contabiliza el nº de podas por cota optimista (estados que se han generado, que no son terminales y que no se incluyen en el conjunto de estados activos porque no pueden dar lugar a soluciones mejores que la que tenemos hasta el momento).

Esas variables ya están declaradas y se muestra el resultado por salida estándar. Falta actualizar estas variables donde corresponda.

<a id='codigo'></a>

# Código a completar

Debes realizar las 3 actividades anteriores en la siguiente celda de código:

In [14]:
def ensamblaje(costes, verbosity=1, initial='greedy1'):
    """
    costes es una matriz numpy de talla MxM con valores positivos

    costes[i,j] es el coste de ensamblar la pieza i cuando ya se han
    ensamblado j piezas.

    Devuelve la mejor solución (una de ellas si hay empates) y su
    valor.
    """

    # no haría falta pero por si acaso comprobamos que costes es una
    # matriz numpy cuadrada y de costes positivos
    assert(type(costes) is np.ndarray and len(costes.shape) == 2
           and costes.shape[0] == costes.shape[1] and costes.dtype == np.int32
           and costes.min()>=0)

    # variables accesibles desde las funciones/clausuras definidas
    # dentro de ensamblaje:
    M = costes.shape[0]
    # la forma más barata de ensamblar la pieza i si podemos elegir el
    # momento de ensamblaje que más nos convenga:
    minCoste = [costes[i,:].min() for i in range(M)]
    # .min() es un método de numpy, también serviría min(costes[:,j])
    if verbosity>1:
        print("minCoste:", minCoste)

    def branch(s_score, s):
        """
        s_score es el score de s
        s es una solución parcial
        """
        for j in range(M): # todos los instantes
            if j not in s: 
                new_score = s_score - minCoste[len(s)] + costes[len(s),j]
                yield (new_score, s + [j])

    def is_complete(s):
        """
        s es una solución parcial
        """
        return len(s) == M

    def naive_solution():
        score, solution = 0, []
        for i in range(M):
            solution.append(i)
            score += costes[i,i]
        return score,solution        

    def greedy_solution1():
        """
        En este caso vamos fila por fila y elegimos la columna de
        menor valor de entre las que queden disponibles.
        """
        score, solution, chosen = 0, [None]*len(costes), set()
        for i in range(M):
            subset = [j for j in range(M) if j not in chosen]
            chosen.add(subset[np.argmin(costes[i,subset])])
            solution[i] = subset[np.argmin(costes[i,subset])]
            score = score + costes[i,subset[np.argmin(costes[i,subset])]]
        return int(score), solution
        
    def greedy_solution2():
        """
        Algoritmo voraz para inicializar a una "solución cualquiera"
        (la mejor que se pueda en un tiempo razonable)

        En este caso vamos columna por columna y elegimos la fila de
        menor valor de entre las que queden disponibles.
        """
        score, solution, chosen = 0, [None]*M, set()
        for i in range(M):
            subset = [j for j in range(M) if j not in chosen]
            chosen.add(subset[np.argmin(costes[subset, i])])
            solution[i] = subset[np.argmin(costes[subset, i])]
            score = score + costes[subset[np.argmin(costes[subset, i])],i]
        return int(score), solution

    def greedy_solution3():
        """
        Ordenar de menor a mayor todos los valores de la matriz de costes recordando sus coordenadas.
        Después se recorre utilizando los valores que correspondan a piezas e instantes válidos.
        """
        M = costes.shape[0] # nº piezas

        # Inicializar la solución con -1
        solution = [-1] * M
        solution_aux = [-1] * M
        filas = [-1] * M
        cols = [-1] * M
        min=100000
        # Ordenar las tareas por coste
        #sorted_tasks = sorted(range(M), key=lambda i: costes[i,0])
        # Iterar a través de las tareas ordenadas por coste
        for k in range(M):
            for i in range(M):
                for j in range(M):
                    # Encuentra el intervalo de tiempo con el menor coste para la tarea actual
                    if(costes[i,j]<min and i not in filas and j not in cols):
                        min=costes[i,j]
                        col=j
                        fila=i
            min=10000
            filas[k]=fila
            cols[k]=col 
            # Asignar la tarea actual al intervalo de tiempo con el menor coste
            solution_aux[k]=(fila,col)

        solution_aux=sorted(solution_aux, key=lambda i: i[0])

        for i in range(M):
            solution[i]=solution_aux[i][1]

        # Calcular el coste total de la solución
        return ((sum(int(costes[pieza,instante])
        for pieza,instante in enumerate(solution)), solution))
        
    def greedy_solution4():
        """
        La mejor de las tres
        """
        return sorted([greedy_solution1(), greedy_solution2(), greedy_solution3()])[0]

    if initial == 'greedy1':
        initial_solution = greedy_solution1
    elif initial == 'greedy2':
        initial_solution = greedy_solution2
    elif initial == 'greedy3':
        initial_solution = greedy_solution3
    elif initial == 'greedy4':
        initial_solution = greedy_solution4
    elif initial == 'naif':
        initial_solution = naive_solution
    else:
        raise Exception('initial option not supported')

    A = [] # cola de prioridad usando heapq
    fx, x = initial_solution() # inicializamos la mejor solución hasta el momento
    if verbosity > 0:
        print(f"Solución inicial ({initial}):",x,"de coste",fx)

    # añadimos el estado inicial:
    heapq.heappush(A, (sum(minCoste), []) )

    iterations = 0 # nº iteraciones
    gen_states = 0 # nº estados generados
    podas_opt  = 0 # nº podas por cota optimista
    maxA = 0 # tamaño máximo alzanzado por lista de estados activos
    # bucle principal de ramificacion y poda con PODA IMPLICITA
    while len(A)>0 and A[0][0] < fx:
        iterations += 1
        lenA = len(A)
        maxA = max(maxA,lenA)
        s_score, s = heapq.heappop(A)
        if verbosity > 1:
            print(f"Iter. {iterations:05} |A|={lenA:05} max|A|={maxA:05}"+
                  f" fx={fx:04} len(s)={len(s):02} score_s={s_score:04}")
        for child_score, child in branch(s_score, s):
            gen_states += 1
            if is_complete(child): # si es terminal
                # es factible (pq branch solo genera factibles)
                # falta ver si mejora la mejor solucion en curso
                if child_score < fx:
                    if verbosity > 0:
                        print("MEJORAMOS",x, fx, "CON", child,child_score)
                    fx, x = child_score, child
            else: # no es terminal
                # lo metemos en el cjt de estados activos si supera
                # la poda por cota optimista:
                if child_score < fx:
                    heapq.heappush(A, (child_score, child) )
                else:
                    podas_opt += 1
    if verbosity > 0:
        print(f"{iterations} iteraciones, max|A|={maxA}, "+
              f"estados_generados={gen_states}, estados_podados={podas_opt}")
    return x, fx

## Prueba con una matriz conocida

In [15]:
prueba = np.array([[4, 4, 5, 3],
                   [2, 8, 9, 1],
                   [6, 9, 6, 3],
                   [4, 6, 7, 7]],dtype=np.int32)
ensamblaje(prueba, initial='greedy1', verbosity=2)

minCoste: [np.int32(3), np.int32(1), np.int32(3), np.int32(4)]
Solución inicial (greedy1): [3, 0, 2, 1] de coste 17
Iter. 00001 |A|=00001 max|A|=00001 fx=0017 len(s)=00 score_s=0011
Iter. 00002 |A|=00004 max|A|=00004 fx=0017 len(s)=01 score_s=0011
Iter. 00003 |A|=00004 max|A|=00004 fx=0017 len(s)=01 score_s=0012
Iter. 00004 |A|=00004 max|A|=00004 fx=0017 len(s)=02 score_s=0012
Iter. 00005 |A|=00004 max|A|=00004 fx=0017 len(s)=01 score_s=0012
Iter. 00006 |A|=00005 max|A|=00005 fx=0017 len(s)=02 score_s=0012
Iter. 00007 |A|=00006 max|A|=00006 fx=0017 len(s)=02 score_s=0012
Iter. 00008 |A|=00006 max|A|=00006 fx=0017 len(s)=02 score_s=0013
Iter. 00009 |A|=00007 max|A|=00007 fx=0017 len(s)=03 score_s=0013
MEJORAMOS [3, 0, 2, 1] 17 CON [1, 0, 3, 2] 16
Iter. 00010 |A|=00006 max|A|=00007 fx=0016 len(s)=01 score_s=0013
Iter. 00011 |A|=00007 max|A|=00007 fx=0016 len(s)=02 score_s=0013
Iter. 00012 |A|=00006 max|A|=00007 fx=0016 len(s)=02 score_s=0014
Iter. 00013 |A|=00006 max|A|=00007 fx=0016 len

([1, 3, 2, 0], np.int32(15))

Si los cambios realizados son correctos (y nosotros tampoco nos hemos equivocado por nuestra parte), el resultado de la ejecución anterior sería:

```python
minCoste: [3, 1, 3, 4]
MEJORAMOS [3, 0, 2, 1] 17 CON [1, 0, 3, 2] 16
MEJORAMOS [1, 0, 3, 2] 16 CON [1, 3, 2, 0] 15
16 iteraciones, max|A|=7, estados_generados=33, estados_podados=11
```

y el resultado devuelto sería:

```python
([1, 3, 2, 0], 15)
```

Compara los valores reportados con la siguiente ejecución que inicializa la mejor solución con la solución *naif*:

In [16]:
ensamblaje(prueba, initial='naif', verbosity=2)

minCoste: [np.int32(3), np.int32(1), np.int32(3), np.int32(4)]
Solución inicial (naif): [0, 1, 2, 3] de coste 25
Iter. 00001 |A|=00001 max|A|=00001 fx=0025 len(s)=00 score_s=0011
Iter. 00002 |A|=00004 max|A|=00004 fx=0025 len(s)=01 score_s=0011
Iter. 00003 |A|=00006 max|A|=00006 fx=0025 len(s)=01 score_s=0012
Iter. 00004 |A|=00008 max|A|=00008 fx=0025 len(s)=02 score_s=0012
Iter. 00005 |A|=00009 max|A|=00009 fx=0025 len(s)=01 score_s=0012
Iter. 00006 |A|=00011 max|A|=00011 fx=0025 len(s)=02 score_s=0012
Iter. 00007 |A|=00012 max|A|=00012 fx=0025 len(s)=02 score_s=0012
Iter. 00008 |A|=00013 max|A|=00013 fx=0025 len(s)=02 score_s=0013
Iter. 00009 |A|=00014 max|A|=00014 fx=0025 len(s)=03 score_s=0013
MEJORAMOS [0, 1, 2, 3] 25 CON [1, 0, 3, 2] 16
Iter. 00010 |A|=00013 max|A|=00014 fx=0016 len(s)=01 score_s=0013
Iter. 00011 |A|=00014 max|A|=00014 fx=0016 len(s)=02 score_s=0013
Iter. 00012 |A|=00013 max|A|=00014 fx=0016 len(s)=02 score_s=0014
Iter. 00013 |A|=00013 max|A|=00014 fx=0016 len(s)

([1, 3, 2, 0], np.int32(15))

¿Influye la solución inicial en el número de estados generados? ¿por qué?

## Prueba con generador de instancias

A continuación se utiliza la función `genera_instancia` definida al inicio del boletín y que nos permite generar instancias de tallas que puedes controlar para realizar experimentos y comparar el uso de la función `

In [17]:
costes = genera_instancia(M=20, high=1000)
for ini in ('naif', 'greedy1', 'greedy2', 'greedy3'):
    #print(f"Con inicialización {ini}:")
    sol = ensamblaje(costes, verbosity=1, initial=ini)
    print(sol)
    print("\n")

# nota: las soluciones no necesariamente han de coincidir (podría haber empates), pero los costes asociados sí

Solución inicial (naif): [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19] de coste 10319
MEJORAMOS [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19] 10319 CON [1, 19, 0, 7, 15, 14, 12, 8, 13, 4, 18, 9, 6, 10, 11, 16, 17, 2, 3, 5] 2161
MEJORAMOS [1, 19, 0, 7, 15, 14, 12, 8, 13, 4, 18, 9, 6, 10, 11, 16, 17, 2, 3, 5] 2161 CON [1, 19, 3, 5, 15, 14, 12, 8, 13, 4, 7, 9, 6, 10, 11, 16, 17, 2, 18, 0] 2096
MEJORAMOS [1, 19, 3, 5, 15, 14, 12, 8, 13, 4, 7, 9, 6, 10, 11, 16, 17, 2, 18, 0] 2096 CON [1, 19, 0, 5, 15, 2, 12, 8, 13, 4, 7, 9, 6, 10, 11, 16, 17, 18, 3, 14] 2068
MEJORAMOS [1, 19, 0, 5, 15, 2, 12, 8, 13, 4, 7, 9, 6, 10, 11, 16, 17, 18, 3, 14] 2068 CON [1, 19, 3, 5, 15, 14, 12, 8, 13, 4, 7, 9, 6, 10, 11, 16, 0, 2, 18, 17] 1548
530875 iteraciones, max|A|=2268315, estados_generados=4521679, estados_podados=1786797
([1, 19, 3, 5, 15, 14, 12, 8, 13, 4, 7, 9, 6, 10, 11, 16, 0, 2, 18, 17], np.int32(1548))


Solución inicial (greedy1): [1, 19, 3, 7, 15, 

## Volvemos a probar

In [18]:
costes = genera_instancia(M=20, high=1000)
for ini in ('naif', 'greedy1', 'greedy2', 'greedy3'):
    #print(f"Con inicialización {ini}:")
    sol = ensamblaje(costes, verbosity=1, initial=ini)
    print(sol)
    print("\n")

# nota: las soluciones no necesariamente han de coincidir (podría haber empates), pero los costes asociados sí

Solución inicial (naif): [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19] de coste 10024
MEJORAMOS [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19] 10024 CON [12, 15, 11, 10, 9, 5, 18, 4, 8, 0, 6, 19, 3, 13, 17, 7, 1, 2, 16, 14] 1960
MEJORAMOS [12, 15, 11, 10, 9, 5, 18, 4, 8, 0, 6, 19, 3, 13, 17, 7, 1, 2, 16, 14] 1960 CON [10, 15, 11, 14, 9, 12, 18, 4, 8, 0, 6, 19, 3, 13, 17, 7, 1, 2, 16, 5] 1921
MEJORAMOS [10, 15, 11, 14, 9, 12, 18, 4, 8, 0, 6, 19, 3, 13, 17, 7, 1, 2, 16, 5] 1921 CON [10, 15, 11, 14, 9, 5, 18, 4, 8, 0, 6, 19, 3, 13, 17, 7, 1, 2, 16, 12] 1343
1391670 iteraciones, max|A|=7903807, estados_generados=12531853, estados_podados=3300512
([10, 15, 11, 14, 9, 5, 18, 4, 8, 0, 6, 19, 3, 13, 17, 7, 1, 2, 16, 12], np.int32(1343))


Solución inicial (greedy1): [10, 15, 9, 1, 3, 12, 18, 4, 2, 0, 6, 19, 8, 13, 7, 11, 16, 17, 5, 14] de coste 3241
MEJORAMOS [10, 15, 9, 1, 3, 12, 18, 4, 2, 0, 6, 19, 8, 13, 7, 11, 16, 17, 5, 14] 3241 CON [12, 15